Реализовать функцию, которая выполняет стэкинг вариант 2 (см. лекцию):  
stack_pred(estimator, X, y, Xt, k, method)  
return sX, sXt  
estimator – sklearn classifier или regressor  
method может быть ’predict’ или ’predict_proba’  
X, y, Xt, sX, sXt – numpy.array  
k – количество фолдов, для разбиения на фолды  
использовать: sklearn.model_selection.KFold

In [98]:
import numpy
import pandas

import sklearn.metrics
import sklearn.datasets
import sklearn.model_selection

import sklearn.tree
import sklearn.linear_model
import sklearn.ensemble

import matplotlib.pyplot as plt
%matplotlib inline

In [99]:
X_full, y_full = sklearn.datasets.load_breast_cancer(return_X_y=True)

In [100]:
X, Xt, y, yt = sklearn.model_selection.train_test_split(X_full, y_full, test_size=0.25, random_state=0)

In [101]:
import numpy as np
rez = [3] * 5
print(rez)

# функция кладет элементы на нужные места в списке
def put_on_places(arr, elems, places):
    for i in range(0, len(elems)):
        arr[places[i]] = elems[i]

put_on_places(rez, [1,2], [4,1])            
print(rez)

[3, 3, 3, 3, 3]
[3, 2, 3, 3, 1]


In [102]:
arr_predict = np.array([[0, 0, 0],
                        [1, 0, 0],
                        [1, 1, 0],
                        [1, 1, 1]])

arr_predict_proba = np.array([[np.array([0, 1]), np.array([1, 1]), np.array([0, 1])],
                              [np.array([0, 0]), np.array([1, 1]), np.array([0, 1])],
                              [np.array([0, 0]), np.array([1, 1]), np.array([0, 1])],
                              [np.array([0, 0]), np.array([0, 1]), np.array([0, 1])]])

# функция производит голосование по столбцам ndarray, работает и для predict, и для predict_proba
def voting(arr):
    axis = 0
    u, indices = np.unique(arr, return_inverse=True)
    return u[np.argmax(np.apply_along_axis(np.bincount, axis, indices.reshape(arr.shape),
                                None, np.max(indices) + 1), axis=axis)]
    
print(voting(arr_predict))
print('---')
print(voting(arr_predict_proba))

[1 0 0]
---
[[0 0]
 [1 1]
 [0 1]]


In [103]:
# В целом верно, но есть замечания:
# 1. В задании предполагалось, что метод будет передан строкой, у вас же в виде clf.predict - это плохая практика
# 2. Вы используете список sX и функцию put_on_places для расстановки элементов на правильные позиции, 
# это работает намного медленнее, чем слайсинг numpy, то же самое можно было сделать следующим образом с использованием слайсинга:
# sX = numpy.zeros(X.shape[0]) вместо sX = [3] * len(X)
# sX[test_index] = pred вместо put_on_places(sX, pred, test_index)

import sklearn.model_selection
import sklearn.tree

def stack_pred(estimator, X, y, Xt, k, method):
    def call_method(estimator, method, vec):
        if (method == 'predict'):
            return estimator.predict(vec)
        elif (method == 'predict_proba'):
            return estimator.predict_proba(vec)
        else:
            raise Exception('unknown method=' + method + '!')
    
    if (method == 'predict'):
        sX = np.zeros(X.shape[0])
    else:
        sX = [3] * len(X)
            
    sXt = []
    kf = sklearn.model_selection.KFold(n_splits=k, shuffle=True, random_state=0)
    for train_index, test_index in kf.split(X):
        # sX - предсказания, полученные для k-го фолда при обучении на k-1м фолде
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        estimator.fit(X_train, y_train)
        pred = call_method(estimator, method, X_test)
        
        if (method == 'predict'):
            sX[test_index] = pred
        else:
            put_on_places(sX, pred, test_index)
            
        # sXt - на каждой итерации делаем предсказания для всей тестовой выборки, чтобы потом проголосовать или усреднить
        pred_t = call_method(estimator, method, Xt)
        sXt.append(pred_t)
    # определяем классификатор или регрессор
    if (isinstance(estimator, sklearn.base.ClassifierMixin)):
        sXt = voting(np.array(sXt))
    else: 
        sXt = np.mean(np.array(sXt), axis=0)
    
    return np.array(sX), sXt

clf = sklearn.tree.DecisionTreeClassifier(random_state=0, min_samples_split=8, min_samples_leaf=4)
rez_s_X, rez_s_X_t = stack_pred(clf, X, y, Xt, 3, 'predict')
print(rez_s_X)
print(rez_s_X_t)

[1. 1. 0. 1. 0. 1. 1. 1. 0. 0. 1. 1. 0. 1. 1. 1. 0. 0. 1. 1. 0. 1. 0. 0.
 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 0. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1.
 0. 0. 1. 1. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 0. 0. 1. 1. 1. 1. 1. 0.
 1. 0. 1. 0. 1. 0. 1. 0. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 0. 1. 1. 1. 0. 1.
 1. 0. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 1. 0. 0. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.
 1. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 0. 0.
 0. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 0. 1. 0. 1. 1. 1. 1.
 0. 0. 1. 1. 1. 0. 1. 0. 0. 1. 0. 1. 1. 1. 0. 1. 1. 0. 1. 1. 0. 0. 1. 1.
 1. 0. 0. 0. 1. 0. 1. 1. 1. 0. 1. 0. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 1.
 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0. 0. 1. 1.
 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 0. 0.
 1. 0. 1. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 0. 0. 1. 1. 1. 1. 0. 1. 1.
 0. 1. 1. 1. 0. 1. 1. 1. 0. 0. 0. 1. 1. 0. 1. 1. 1.